# Ruteo

## Preparación de la Sesión

In [2]:
!git clone https://github.com/JuanCruzC97/ope3-logistica.git &> /dev/null

In [3]:
cd ope3-logistica

/content/ope3-logistica


In [4]:
!pip install -e . &> /dev/null

## Módulos

In [5]:
import pandas as pd
import numpy as np
import plotly.express as px

In [6]:
from logistica.ruteo import Ruteo
from logistica.utils import load_inputs, preparar_df_pedidos

In [7]:
from logistica import metaheuristicas as mh

# Carga de Datos

## Subir Datos Externos

In [30]:
# Solo correr esta celda si importamos datos externos de pedidos y camiones.
from google.colab import files
uploaded = files.upload()

Saving data_inputs.xlsx to data_inputs.xlsx


In [31]:
!rm /content/ope3-logistica/data/inputs/data_inputs.xlsx
!mv data_inputs.xlsx /content/ope3-logistica/data/inputs/data_inputs.xlsx

## Generación de Datasets

In [32]:
df_clientes = load_inputs("clientes")
df_pedidos = load_inputs("pedidos")
df_pedidos = pd.merge(left=df_pedidos, right=df_clientes, on="cliente", how="inner")
df_pedidos

,cliente,pedidos1,pedidos2,pedidos3,pedidos4,pedidos5,pedidos6,coord_x,coord_y
0,A,4,3,5,2,6,6,0.257571,1.803726
1,B,6,6,5,4,1,3,1.523313,2.102301
2,C,1,6,6,2,6,7,0.710910,2.629800
3,D,8,6,5,2,8,2,1.013940,2.149740
4,E,4,1,2,4,2,3,0.609480,0.666810
5,F,3,1,6,8,6,5,0.998640,1.451790
6,G,3,2,3,7,4,3,1.434510,0.018180
7,H,5,8,5,2,4,8,2.171880,0.764910
8,I,5,2,3,4,2,6,1.412280,0.863190
9,J,1,8,6,8,1,2,2.386260,1.387260


In [33]:
df_camiones = load_inputs("camiones")
df_camiones

,camion,carga_max,pedidos_max,dist_max
0,1,10,3,2
1,2,10,3,2
2,3,10,3,2
3,4,10,3,2
4,5,10,3,2
5,6,10,3,2


# Optimización 1 Pedido


In [34]:
#@title Seleccionar Pedido
pedidos = 'pedidos6' #@param ["pedidos1", "pedidos2", "pedidos3", "pedidos4", "pedidos5", "pedidos6"]

In [35]:
df_pedido = preparar_df_pedidos(df_pedidos, pedidos)
df_pedido

,cliente,pedidos,coord_x,coord_y
0,A,6,0.257571,1.803726
1,B,3,1.523313,2.102301
2,C,7,0.710910,2.629800
3,D,2,1.013940,2.149740
4,E,3,0.609480,0.666810
5,F,5,0.998640,1.451790
6,G,3,1.434510,0.018180
7,H,8,2.171880,0.764910
8,I,6,1.412280,0.863190
9,J,2,2.386260,1.387260


In [36]:
#@title Parámetros del Ruteo
costo_oportunidad = 3000 #@param {type:"number"}
presupuesto = 1220 #@param {type:"number"}

In [37]:
ruteo = Ruteo(df_camiones, df_pedido, costo_oportunidad=costo_oportunidad, presupuesto=presupuesto)

In [38]:
#@title Parámetros Solución Inicial
mode = 'random' #@param ["simple", "random"]
random_state = None #@param {type:"number"}

In [39]:
ruteo.get_solucion_inicial(mode=mode, random_state=random_state)
ruteo

--Ruteo--
Carga Total: 54tn
Costo Camiones: 58800$
Costo Oportunidad: 21000$ 
Costo Total: 79800$ 
Costo Total por tn: 1477.78$/tn 
Ahorro: 21.13%

In [40]:
#@title Parámetros Metaheurística
temp_inicial = 100 #@param {type:"number"}
temp_final = 1 #@param {type:"number"}
k = 100 #@param {type:"number"}
iters = 10 #@param {type:"number"}
temp_mode = 'non_linear' #@param ["linear", "non_linear"]
p = 1 #@param {type:"number"}
random_state = None #@param {type:"number"}

In [41]:
best_sol, history = mh.sa(ruteo, 
                          t_inicial=temp_inicial, 
                          t_final=temp_final, 
                          k=k, 
                          iters=iters, 
                          temp_mode=temp_mode, 
                          prob=p, 
                          random_state=random_state)
print("\n")
best_sol.summary_ruteo(time=history.get("time"), iters=history.get("iters"))

100%|██████████| 100/100 [00:01<00:00, 93.61it/s]

,
Carga Total,60.00
Costo Camiones,60000.00
Costo Oportunidad,3000.00
Costo Total,63000.00
Costo Total por tn,1050.00
Ahorro,-13.93
Tiempo,1.08
Iteraciones,1000.00


In [42]:
best_sol.summary_camiones()

,Cliente 1,Cliente 2,Cliente 3,Costo,Carga,Costo_tn,Ahorro
Camion 1,J,H,None,10000,10,1000.0,-18.03
Camion 2,A,O,None,10000,10,1000.0,-18.03
Camion 3,D,F,E,10000,10,1000.0,-18.03
Camion 4,K,L,None,10000,10,1000.0,-18.03
Camion 5,I,N,G,10000,10,1000.0,-18.03
Camion 6,C,B,None,10000,10,1000.0,-18.03
Costo Oportunidad,,,,3000,1,,
Total,,,,63000,60,1050.0,-13.93


In [43]:
best_sol.summary_pedidos()

,Carga,Asignado,Camion
Pedido A,6,True,2.0
Pedido B,3,True,6.0
Pedido C,7,True,6.0
Pedido D,2,True,3.0
Pedido E,3,True,3.0
Pedido F,5,True,3.0
Pedido G,3,True,5.0
Pedido H,8,True,1.0
Pedido I,6,True,5.0
Pedido J,2,True,1.0


In [ ]:
mh.make_history_plots(history)

In [ ]:
mh.get_history_df(history)

In [ ]:
ruteo.plot_solution()
best_sol.plot_solution()